In [2]:
from datetime import datetime
import random
from math import radians, sin, cos, sqrt, atan2

In [3]:
def calculate_distance(lat1, lon1, lat2, lon2): #거리 계산 해주는
    R = 6371  # 지구의 반경(km)

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c

    return distance

In [4]:
def route(doctor_location, patients_data, emergency_calls):
    # 1. 5km 반경 내 환자 필터링
    nearby_patients = []
    for patient in patients_data:
        distance = calculate_distance(
            doctor_location[0], doctor_location[1],
            patient['location'][0], patient['location'][1]
        )
        if distance <= 5:
            nearby_patients.append(patient)

    # 2. 긴급 호출 환자 리스트 생성
    emergency_patients = [p for p in nearby_patients if p['patientid'] in emergency_calls]

    # 3. 남은 기간으로 정렬
    nearby_patients.sort(key=lambda x: x['remaining_days'])


    # 현재 날짜를 시드로 설정
    current_date = datetime.now()
    random.seed(current_date.year * 10000 + current_date.month * 100 + current_date.day)

    # 4~7 사이의 랜덤 숫자 선택
    global total_patients_needed
    total_patients_needed = random.randint(4, 7)

    # 4. 경로 계획
    if len(emergency_patients) >= total_patients_needed: #오늘 받는 환자 수 이상의 긴급환자가 있을때
        return emergency_patients # 최종경로는 긴급환자

    if len(emergency_patients) > 0:
        # 긴급 환자가 있는 경우
        remaining_slots = total_patients_needed - len(emergency_patients)
        regular_patients = [p for p in nearby_patients if p['patientid'] not in emergency_calls]
        selected_regular_patients = regular_patients[:remaining_slots]
        return emergency_patients + selected_regular_patients
    else:
        # 긴급 환자가 없는 경우
        return nearby_patients[:total_patients_needed]


In [5]:
def calc_route_distance(route_plan, doctor_location):
    route_distance = []
    #의사와 첫번째 환자 사이의 거리 계산
    route_distance.append(calculate_distance(
            doctor_location[0], doctor_location[1],
            route_plan[0]['location'][0], route_plan[0]['location'][1]
        ))
    # 두번째 환자부터 각 환자 사이의 거리를 계산하여 추가
    for i in range(1,total_patients_needed-1):
        current_location = route_plan[i]['location']
        next_location = route_plan[i+1]['location']
        distance = calculate_distance(
            current_location[0], current_location[1],
            next_location[0], next_location[1]
        )
        route_distance.append(distance)

    return route_distance


In [7]:
from geopy.geocoders import Nominatim

# Geolocator 객체 생성
geolocator = Nominatim(user_agent="South Korea")

# 주소
address = "경기도 성남시 수정구 태평동 4063"

# 주소로부터 위도와 경도를 얻음
location = geolocator.geocode(address)

# 결과 출력
if location:
    print(f"Address: {address}")
    print(f"Latitude: {location.latitude}, Longitude: {location.longitude}")
else:
    print("Location not found.")


Address: 경기도 성남시 수정구 태평동 4063
Latitude: 37.446433, Longitude: 127.1319544


In [8]:
# 테스트를 위한 메인 부분 추가
if __name__ == "__main__":
    # 의사 현재 위치 (서울시청 좌표)
    doctor_loc = [37.4464335, 127.1319544]

    # 환자 데이터 예시
    patients = [
        {'patientid': 1, 'remaining_days': -28, 'location': [37.449399149433056, 127.14601526356022]},
        {'patientid': 2, 'remaining_days': -115, 'location': [37.4380831, 127.1444545]},
        {'patientid': 3, 'remaining_days': 39, 'location': [37.433176613176435, 127.14161080088701]},
        {'patientid': 4, 'remaining_days': 101, 'location': [37.43021647283937, 127.1451237800741]},
        {'patientid': 5, 'remaining_days': -131, 'location': [37.43901598293697, 127.14499567791044]},
        {'patientid': 6, 'remaining_days': -93, 'location': [37.44169056961609, 127.13965915647442]},
        {'patientid': 7, 'remaining_days': -109, 'location': [37.43267341052862, 127.14186008911558]},
        {'patientid': 8, 'remaining_days': 41, 'location': [37.4410347, 127.131485]},
        {'patientid': 9, 'remaining_days': -96, 'location': [37.43433366005426, 127.14145748387472]},
        {'patientid': 10, 'remaining_days': 31, 'location': [37.43250915, 127.14397198375899]},
        {'patientid': 11, 'remaining_days': 23, 'location': [37.43163931627512, 127.1352877171919]},
        {'patientid': 12, 'remaining_days': 32, 'location': [37.430780602569136, 127.13167955503987]},
        {'patientid': 13, 'remaining_days': -253, 'location': [37.43850724401532, 127.1416294547357]},
        {'patientid': 14, 'remaining_days': -158, 'location': [37.4316062, 127.1284259]},
        {'patientid': 15, 'remaining_days': 32, 'location': [37.44013932601631, 127.14167111769201]}
    ]

    emergency_calls = [2, 4]

    # route 함수 호출 및 결과 출력
    route_plan = route(doctor_loc, patients, emergency_calls)
    route_distance = calc_route_distance(route_plan,doctor_loc)
    print("\n=== 방문 계획 ===")
    print(f"총 방문 환자 수: {len(route_plan)}명")
    print("\n방문 순서:")
    for i, patient in enumerate(route_plan, 1):
        print(f"{i}번째 방문: ID {patient['patientid']}, 남은 일수: {patient['remaining_days']}, 위치: {patient['location']}")
    print("\n방문 거리")
    i = 1
    for nth_distance in route_distance:
        if i == 1 :
            print("의사와",i,"번째 환자 사이의 거리는",nth_distance,"입니다.")
            i += 1
        else :
            print(i,"번째 환자와",i+1,"번째 환자 사이의 거리는", nth_distance,"입니다.")
            i += 1


=== 방문 계획 ===
총 방문 환자 수: 6명

방문 순서:
1번째 방문: ID 2, 남은 일수: -115, 위치: [37.4380831, 127.1444545]
2번째 방문: ID 4, 남은 일수: 101, 위치: [37.43021647283937, 127.1451237800741]
3번째 방문: ID 13, 남은 일수: -253, 위치: [37.43850724401532, 127.1416294547357]
4번째 방문: ID 14, 남은 일수: -158, 위치: [37.4316062, 127.1284259]
5번째 방문: ID 5, 남은 일수: -131, 위치: [37.43901598293697, 127.14499567791044]
6번째 방문: ID 7, 남은 일수: -109, 위치: [37.43267341052862, 127.14186008911558]

방문 거리
의사와 1 번째 환자 사이의 거리는 1.4422288593961996 입니다.
2 번째 환자와 3 번째 환자 사이의 거리는 0.9721494548781217 입니다.
3 번째 환자와 4 번째 환자 사이의 거리는 1.39567400704018 입니다.
4 번째 환자와 5 번째 환자 사이의 거리는 1.6790557041065468 입니다.
5 번째 환자와 6 번째 환자 사이의 거리는 0.7576541752205803 입니다.


# 지도 시각화

In [9]:
pip install folium


In [15]:
import folium

patients = [
    {"ID": 2, "remaining_days": -115, "location": [37.4380831, 127.1444545]},
    {"ID": 4, "remaining_days": 101, "location": [37.43021647283937, 127.1451237800741]},
    {"ID": 13, "remaining_days": -253, "location": [37.43850724401532, 127.1416294547357]},
    {"ID": 14, "remaining_days": -158, "location": [37.4316062, 127.1284259]},
    {"ID": 5, "remaining_days": -131, "location": [37.43901598293697, 127.14499567791044]},
    {"ID": 7, "remaining_days": -109, "location": [37.43267341052862, 127.14186008911558]},
]


# 지도 초기화 (서울 중심)
m = folium.Map(location=[37.446433, 127.1319544], zoom_start=14)

# 마커 위치 리스트 (선 그리기 위해)
locations = []

# 방문 순서대로 마커 추가
for patient in patients:
    folium.Marker(
        location=patient['location'],  # 위치
        popup=f"ID: {patient['ID']}<br>Remaining Days: {patient['remaining_days']}",  # 팝업 내용
        icon=folium.Icon(color='blue', icon='info-sign')  # 마커 아이콘
    ).add_to(m)

    # 위치 리스트에 추가 (마커들 연결을 위해)
    locations.append(patient['location'])

# 마커들 이어주는 선 추가
folium.PolyLine(locations, color='blue', weight=2.5, opacity=1).add_to(m)


In [16]:
m